In [192]:
from collections import Counter

In [ ]:
def read_file(filename='test'):
    input_file = open(f'data/{filename}_24', 'r')
    data = input_file.read().splitlines()
    data = [",".join(r.split("@")) for r in data]
    data = [[int(v) for v in r.split(",")] for r in data]
    res = []
    for d in data:
        res.append({"x":d[0], "y":d[1], "z":d[2], "dx":d[3], "dy":d[4], "dz":d[5]})
    return res

In [ ]:
def calc_formula_2d(r):
    a = r["dy"]/r["dx"]
    b = r["y"]+r["dy"] - a*(r["x"]+r["dx"])
    return (a,b)

def is_in_past_2d(r, crossing):
    x_diff = (r["x"] - crossing["x"]) >= 0
    y_diff = (r["y"] - crossing["y"]) >= 0
    x_dir = r["dx"] >= 0
    y_dir = r["dy"] >= 0
    
    return (x_diff == x_dir) and (y_diff == y_dir)

def find_intersections_2d(data):
    intersections_past = []
    intersections_future = []
    parallels = []
    for h1 in range(len(data)):
        for h2 in range(h1+1, len(data)):
            if data[h1]['a'] == data[h2]['a']:
                parallels.append((h1,h2))
                continue
            cross_x = (data[h2]['b'] - data[h1]['b'])/(data[h1]['a'] - data[h2]['a'])
            cross_y = data[h1]['a']*cross_x + data[h1]['b']
            if is_in_past_2d(data[h1], {"x":cross_x, "y":cross_y}) | is_in_past_2d(data[h2], {"x":cross_x, "y":cross_y}):
                intersections_past.append((h1,h2,cross_x,cross_y))
            else:
                intersections_future.append((h1,h2,cross_x,cross_y))
    return intersections_past,intersections_future,parallels

In [ ]:
def first_star(filename='test'):
    BOUNDRIES_RANGE =  (200000000000000, 400000000000000)
    data = read_file(filename)
    for r in data:
        formula = calc_formula_2d(r)
        r['a'] = formula[0]
        r['b'] = formula[1]
    intersections_past,intersections_future,parallels = find_intersections_2d(data)
    res = 0
    for int_future in intersections_future:
        if (int_future[2] >= BOUNDRIES_RANGE[0]) & (int_future[2] <= BOUNDRIES_RANGE[1]) & (int_future[3] >= BOUNDRIES_RANGE[0]) & (int_future[3] <= BOUNDRIES_RANGE[1]):
            res += 1
    return res

In [215]:
def second_star(filename='test'):
    data = read_file(filename)

    #Get the dimension for which there is more than one rock going exactly in the same coordinate in each timepoint
    known_dim = None
    for dim in ("x","y","z"):
        ctr = Counter([(x[dim],x[f'd{dim}']) for x in data]).most_common(1)
        if ctr[0][1] > 1:
            known_dim = (ctr[0][0], dim)
    res = {}
    res[known_dim[1]] = known_dim[0][0]

    #Caclulate t1, t2 and t3 using the known dimension
    dim = known_dim[1]
    v1  = data[0][dim]
    dv1 = data[0][f"d{dim}"]
    v2  = data[1][dim]
    dv2 = data[1][f"d{dim}"]
    v3  = data[2][dim]
    dv3 = data[2][f"d{dim}"]
    vr = known_dim[0][0]
    dvr = known_dim[0][1]

    t1 = (vr-v1) / (dv1-dvr)
    t2 = (vr-v2) / (dv2-dvr)
    t3 = (vr-v3) / (dv3-dvr)

    #Calculate the other dimensions using the t1, t2 and t3
    for dim in ("x","y","z"):
        if dim in res:
            continue
        v1  = data[0][dim]
        dv1 = data[0][f"d{dim}"]
        v2  = data[1][dim]
        dv2 = data[1][f"d{dim}"]
        v3  = data[2][dim]
        dv3 = data[2][f"d{dim}"]

        dvr = (v2 + dv2*t2 - v1 - dv1*t1)/(t2 - t1)
        vr = v1 + dv1*t1 - t1*dvr

        res[dim] = vr
    return int(sum(res.values()))

In [216]:
first_star('input')

13910

In [217]:
second_star('input')

618534564836937

In [ ]:
#Second star without the trick - could not solve in time - t3 calculated using formula from three rocks. The formula for t1,t2 could also be extracted but it would be very complex
def calculate_times(data, max_range=100000):
    for t1 in range(1,max_range):
        for t2 in range(1,max_range):
            if t1==t2:
                continue
            res = []
            for dim in ("x","y","z"):
                v1  = data[0][dim]
                dv1 = data[0][f"d{dim}"]
                v2  = data[1][dim]
                dv2 = data[1][f"d{dim}"]
                v3  = data[2][dim]
                dv3 = data[2][f"d{dim}"]

                A = (v2 - v1 + dv2*t2 - dv1*t1)/(t2 - t1)
                if dv3-A == 0:
                    continue
                t3 = (v1 + dv1*t1 - t1*A - v3)/(dv3 - A)
                res.append(t3)
            if len(res)==3 and len(set(res))==1:
                return (t1,t2,t3)